<a href="https://www.kaggle.com/code/deepkayastha6890/transaction-risk-pattern-finder?scriptVersionId=297098559" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import kagglehub
from kagglehub import KaggleDatasetAdapter
import os

# Addition paths
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Finding the file and assigning the dataframe column

In [ ]:
file_path = 'bank.xlsx'

df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "apoorvwatsky/bank-transaction-data",
    file_path,
)

# Dataframe first five rows
print("\n\nFirst 5 records:")
display(df.head())


### In this cell cleaning the null data also removing the no needed columns

In [ ]:
def DataCleaning(df): 
    
    cleaningData = df.copy()

    #Finding the information 
    print("\n\nInformations:\n",cleaningData.info())

    # Finding the null values
    majorColumns = ['DEPOSIT AMT','WITHDRAWAL AMT','DATE','VALUE DATE']
    for i in majorColumns:
        print(f'\n\n{i} : ',cleaningData[i].isnull().sum())

    # Filling the null values
    nullValuesFiller = ['DEPOSIT AMT','WITHDRAWAL AMT','CHQ.NO.']
    for i in nullValuesFiller:
        cleaningData[i] = cleaningData[i].fillna(0)

    # Just changing the date name
    cleaningData['ActualDate'] = cleaningData['VALUE DATE']
    
    # Seperating the day , month , year 
    cleaningData['ActualDay'] = cleaningData['VALUE DATE'].dt.day 
    cleaningData['ActualMonth'] = cleaningData['VALUE DATE'].dt.month
    cleaningData['ActualYear'] = cleaningData['VALUE DATE'].dt.year

    # Sorting the data from starting date
    cleaningData = cleaningData.sort_values(by="VALUE DATE", ascending=False)

    # Droping unuseful data colums [Account No, , . ,CHQ.NO. ]
    cleaningData = cleaningData.drop(columns=['.','CHQ.NO.','VALUE DATE'])
    
    return cleaningData
    
cleanData = DataCleaning(df)
    

In [ ]:
def DataDiscribtion(df):

    dataDiscribe = df.copy()
    # Finding the columns 
    print('\n')
    print(dataDiscribe.columns)

    # finding the shape
    print("\nDataset Size",dataDiscribe.shape)

    # Finding the data discribtion 
    print('\nDiscribtion of the data : ')
    display(dataDiscribe.describe())

    
    # Year wise count
    total_year_counts = dataDiscribe['ActualYear'].value_counts()

    print("Total list of year : ")
    display(total_year_counts.head())
    print("\n")
    
    # Less Then Zero balance
    count =0
    for value in dataDiscribe['BALANCE AMT']:
        if(value < 0):
            count = count +1;
    
    # Counting the minus value
    print("\n\nMinus Value in balance : ",count)
    
    # Finding the data distribution
    print('\n\nUnique value of the account numbers')
    display(dataDiscribe['Account No'].unique())
    
DataDiscribtion(cleanData)


### Actual Dataset have total 10 use full column 
- In the dataset there is multiple account numbers
- Total rows 116201
- total 5 years of data is availble

### Using this two function we can group the dataset

In [ ]:
# Count of compressed features
def groupByCounting(df, respectedMainFeature, copresstionFeatures):
    return (
        df
        .groupby(respectedMainFeature)[copresstionFeatures]
        .count()
        .sort_index()
    )
# Sum of the compress features
def groupBySum(df, respectedMainFeature, copresstionFeatures):
    return (
        df
        .groupby(respectedMainFeature)[copresstionFeatures]
        .sum()
        .sort_index()
    )


# This is the main feature engineering portion using this we have the risk factors and risk of the bank account

In [ ]:
# Day wise data process and account No wise classifier
def build_daily_account_log(df):

    # Day wise agrigated data and this data is days wise
    daily_log = (
        df
        .sort_values(['Account No', 'DATE'])
        .groupby(['Account No', 'DATE'])
        .agg(
            TotalDeposit=('DEPOSIT AMT', 'sum'),
            TotalWithdrawal=('WITHDRAWAL AMT', 'sum'),
            EndBalance=('BALANCE AMT', 'last'),
            TxnCount=('BALANCE AMT', 'count')
        )
        .reset_index()
    )

    # Net movement for the day
    daily_log['NetChange'] = (
        daily_log['TotalDeposit'] - daily_log['TotalWithdrawal']
    )

    # This making diffrent dataframes 
    daily_account_df = daily_log


    # Now daily_account_df is new all in one data frame
    # If overdraf this goes true
    daily_account_df['IsOverdraft'] = daily_account_df['EndBalance'] < 0
    
    # Marking overdraft
    daily_account_df['OverdraftAmount'] = (
        -daily_account_df['EndBalance']
    ).clip(lower=0)

    # Daily balance change
    daily_account_df['DailyBalanceChange'] = (
    daily_account_df
        .groupby('Account No')['EndBalance']
        .diff()
    )
    daily_account_df['MONTH'] = daily_account_df['DATE'].dt.to_period('M')

    # Two dataframes are there
    return daily_log , daily_account_df
_ , manngedData = build_daily_account_log(cleanData)

display(manngedData.shape)
display(manngedData.head())


In [ ]:
def monthly_account_features(df):

    df = df.sort_values(['Account No', 'DATE']).copy()

    # putting the feature risk one row behind
    df['StreakStart'] = (
        (df['IsOverdraft']) &
        (~df.groupby('Account No')['IsOverdraft'].shift(fill_value=False))
    )

    # Assigning the streak id
    df['StreakID'] = df.groupby('Account No')['StreakStart'].cumsum()

    # Counting the overdraft day
    overdraft_days = df[df['IsOverdraft']]

    
    streaks = (
        overdraft_days
        .groupby(['Account No', 'MONTH', 'StreakID'])
        .size()
        .reset_index(name='StreakLength')
    )

    # Avg overdraft streak per month (Recovery Velocity)
    recovery_velocity = (
        streaks
        .groupby(['Account No', 'MONTH'])['StreakLength']
        .mean()
        .reset_index(name='RecoveryVelocity')
    )

    # MONTHLY grouping the data 
    monthly = (
        df
        .groupby(['Account No', 'MONTH'])
        .agg(
            DaysInMonth=('DATE', 'nunique'),
            StressDays=('IsOverdraft', 'sum'),
            NetCashflow=('NetChange', 'sum'),
            TotalTxn=('TxnCount', 'sum'),
            MaxWithdrawal=('TotalWithdrawal', 'max'),
            AvgEndBalance=('EndBalance', 'mean'),
            BalanceVolatility=('EndBalance', 'std')
        )
        .reset_index()
    )

    # 2. Whale Impact
    monthly['WhaleImpact'] = (
        monthly['MaxWithdrawal'] /
        monthly['AvgEndBalance'].abs()
    )

    # 3. Stress Duration
    monthly['StressDuration'] = (
        monthly['StressDays'] / monthly['DaysInMonth']
    )

    # 5. Operational Intensity
    monthly['OperationalIntensity'] = (
        monthly['TotalTxn'] / monthly['DaysInMonth']
    )

    # MERGE RECOVERY VELOCITY
    final_features = monthly.merge(
        recovery_velocity,
        on=['Account No', 'MONTH'],
        how='left'
    )

    final_features['RecoveryVelocity'] = (
        final_features['RecoveryVelocity'].fillna(0)
    )

    return final_features
monthly_features = monthly_account_features(manngedData)
display(monthly_features.head(10))
monthly_features.tail(10)



In [ ]:
def modelDataFrame(df):
    df = df.copy()
    
    df['Norm_Recovery'] = (df['RecoveryVelocity'] / 30.0).clip(upper=1.0)
    df['Norm_Stress'] = df['StressDuration'].clip(upper=1.0)
    df['Norm_Whale'] = df['WhaleImpact'].clip(upper=1.0)
    operating_scale = df[['MaxWithdrawal', 'AvgEndBalance']].abs().max(axis=1)
    df['Norm_Cashflow'] = np.where(
        df['NetCashflow'] >= 0,
        0.0,
        (df['NetCashflow'].abs() / operating_scale).clip(upper=1.0)
    )
    df['Norm_Volatility'] = (
        np.log1p(df['BalanceVolatility']) / np.log1p(df['AvgEndBalance'].abs())
    ).clip(upper=1.0)
    df['Norm_Intensity'] = (df['OperationalIntensity'] / 50.0).clip(upper=1.0)
    relation = df[
        ['Norm_Recovery', 'Norm_Stress', 'Norm_Cashflow',
         'Norm_Whale', 'Norm_Intensity', 'Norm_Volatility']
    ].copy()
    df['Risk_Score'] = relation.mean(axis=1) * 100

    return df.round(2), relation.round(3)


# Execution
modelData, relation = modelDataFrame(monthly_features)

modelData = modelData.sort_values(['Account No', 'MONTH'])
modelData['FutureRisk'] = modelData.groupby('Account No')['Risk_Score'].shift(-1)

train_df = modelData.dropna(subset=['FutureRisk'])
live_df  = modelData[modelData['FutureRisk'].isna()]
relation_corr = relation.copy()
relation_corr['risk'] = modelData['FutureRisk']

corr_matrix = relation_corr.corr(method='spearman')
print(corr_matrix)
modelData.to_csv('myData.csv')
modelData.describe ()

finalData = modelData[
    ['Norm_Recovery', 'Norm_Stress', 'Norm_Cashflow',
     'Norm_Whale', 'Norm_Intensity', 'Norm_Volatility','FutureRisk',
     'Account No','MONTH']  
].copy()


In [ ]:
import matplotlib.dates as mdates

if pd.api.types.is_period_dtype(train_df['MONTH']):
    train_df['MONTH'] = train_df['MONTH'].dt.to_timestamp()
else:
    if not pd.api.types.is_datetime64_any_dtype(train_df['MONTH']):
        train_df['MONTH'] = pd.to_datetime(train_df['MONTH'])

train_df['Account No'] = train_df['Account No'].astype(str)

sns.set_theme(style="whitegrid")
accounts = train_df['Account No'].unique()

plt.figure(figsize=(14, 6))
risk_melt = train_df.melt(id_vars=['Account No', 'MONTH'], 
                          value_vars=['Risk_Score', 'FutureRisk'], 
                          var_name='Risk_Type', value_name='Score')

sns.lineplot(data=risk_melt, x='MONTH', y='Score', hue='Account No', style='Risk_Type', markers=True, dashes=False)
plt.title('Risk Trajectory: Current vs. Predicted Risk (All Accounts)', fontsize=16)
plt.ylabel('Risk Score')
plt.xlabel('Date')
plt.legend(title='Account / Risk Type')
plt.tight_layout()
plt.show()

# account wise net cashflow and risk score
for acc in accounts:
    subset = train_df[train_df['Account No'] == acc]
    
    fig, ax1 = plt.subplots(figsize=(14, 6))
    
    color_cash = 'tab:green'
    ax1.set_xlabel('Month')
    ax1.set_ylabel('Net Cashflow', color=color_cash, fontweight='bold')
    sns.lineplot(data=subset, x='MONTH', y='NetCashflow', ax=ax1, color=color_cash, marker='o', linewidth=2.5)
    ax1.tick_params(axis='y', labelcolor=color_cash)
    ax1.axhline(0, color='black', linestyle='--', linewidth=1) # Zero line
    
    ax2 = ax1.twinx()
    color_stress = 'tab:red'
    ax2.set_ylabel('Stress Days (Count)', color=color_stress, fontweight='bold')
    
    width = 20 
    ax2.bar(subset['MONTH'], subset['StressDays'], color=color_stress, alpha=0.3, width=20, label='Stress Days')
    ax2.tick_params(axis='y', labelcolor=color_stress)
    
    
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.title(f'Account Status: {acc} (Cashflow vs Stress)', fontsize=16, weight='bold')
    
    latest = subset.iloc[-1]
    status_text = "STABLE" if latest['Risk_Score'] < 50 else "AT RISK"
    status_color = "green" if status_text == "STABLE" else "red"
    
    plt.figtext(0.15, 0.8, f"Current Status: {status_text}\nRisk Score: {latest['Risk_Score']:.2f}", 
                bbox=dict(facecolor=status_color, alpha=0.2), fontsize=12)

    plt.tight_layout()
    plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


def train_and_predict_future_risk(finalData):
    df = finalData.copy()

    base_features = [
        'Norm_Recovery',
        'Norm_Stress',
        'Norm_Cashflow',
        'Norm_Whale',
        'Norm_Intensity',
        'Norm_Volatility'
    ]

    needed_cols = base_features + ['FutureRisk', 'Account No', 'MONTH']
    for col in needed_cols:
        if col not in df.columns:
            raise ValueError(f"Missing column in finalData: {col}")

    df = df.dropna(subset=base_features).copy()

    df = df.sort_values(['Account No', 'MONTH']).copy()

    for col in base_features:
        df[f"{col}_lag1"] = df.groupby("Account No")[col].shift(1)
        df[f"{col}_lag2"] = df.groupby("Account No")[col].shift(2)

    lag_cols = [f"{c}_lag1" for c in base_features] + [f"{c}_lag2" for c in base_features]

    df_train = df.dropna(subset=['FutureRisk'] + lag_cols).copy()

    feature_cols = base_features + lag_cols

    split_point = df_train['MONTH'].quantile(0.8)

    train_part = df_train[df_train['MONTH'] <= split_point]
    val_part   = df_train[df_train['MONTH'] > split_point]

    X_train = train_part[feature_cols]
    y_train = train_part['FutureRisk']

    X_val = val_part[feature_cols]
    y_val = val_part['FutureRisk']

    model = XGBRegressor(
        n_estimators=2000,
        learning_rate=0.02,
        max_depth=4,
        subsample=0.85,
        colsample_bytree=0.85,
        min_child_weight=3,
        reg_alpha=0.2,
        reg_lambda=1.5,
        objective="reg:squarederror",
        random_state=42
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    val_preds = model.predict(X_val)

    metrics = {
        "MAE": mean_absolute_error(y_val, val_preds),
        "RMSE": np.sqrt(mean_squared_error(y_val, val_preds)),
        "R2": r2_score(y_val, val_preds)
    }

    print("Metrics:", metrics)

    plt.figure(figsize=(6,6))
    plt.scatter(y_val, val_preds, alpha=0.5)
    plt.plot([y_val.min(), y_val.max()],
             [y_val.min(), y_val.max()],
             linestyle="--")
    plt.xlabel("Actual Future Risk")
    plt.ylabel("Predicted Future Risk")
    plt.title("Predicted vs Actual Future Risk (Validation)")
    plt.show()

    residuals = y_val - val_preds

    plt.figure(figsize=(6,4))
    plt.scatter(val_preds, residuals, alpha=0.5)
    plt.axhline(0, linestyle="--")
    plt.xlabel("Predicted Future Risk")
    plt.ylabel("Residual (Actual − Predicted)")
    plt.title("Residuals vs Predicted Risk")
    plt.show()

    # Feature importance
    importance = pd.Series(model.feature_importances_, index=feature_cols).sort_values()

    plt.figure(figsize=(7,5))
    importance.tail(20).plot(kind="barh")  
    plt.xlabel("Importance")
    plt.title("Top Feature Importance (XGBoost)")
    plt.show()

    df_live = df.copy()

    df_live = df_live.dropna(subset=lag_cols).copy()

    # Latest month per account
    df_live_latest = (
        df_live.sort_values(['Account No', 'MONTH'])
        .groupby("Account No")
        .tail(1)
        .copy()
    )

    df_live_latest["PredictedFutureRisk"] = model.predict(df_live_latest[feature_cols])

    live_predictions = df_live_latest[
        ["Account No", "MONTH", "PredictedFutureRisk"]
    ].reset_index(drop=True)

    return model, metrics, live_predictions

model, metrics, predictions = train_and_predict_future_risk(finalData)

print(metrics)
display(predictions.head(10))
